In [4]:
import numpy as np
import pandas as pd
import cv2
import csv
import json
import os.path
import pickle
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Flatten, Lambda
from keras.optimizers import Adam
import tensorflow as tf

In [5]:
csv_file = 'driving_log.csv'
df = pd.read_csv(csv_file, header=None)
df.columns = ['center','left','right','steering','throttle','brake','speed']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2788 entries, 0 to 2787
Data columns (total 7 columns):
center      2788 non-null object
left        2788 non-null object
right       2788 non-null object
steering    2788 non-null float64
throttle    2788 non-null float64
brake       2788 non-null int64
speed       2788 non-null float64
dtypes: float64(3), int64(1), object(3)
memory usage: 152.5+ KB


In [10]:
df = df[(df.brake==0.0)&(df.speed>0.0)]
df.drop(['left','right','throttle','brake'], axis = 1)
X_train = np.copy(df['center'])
Y_train = np.copy(df['steering'])
Y_train = Y_train.astype(np.float32)